In [1]:
! pip install datasets

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import re
import nltk
import gensim
from gensim.models import word2vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from scipy.sparse import hstack

from gensim.models import Word2Vec

In [3]:
from datasets import get_dataset_split_names

get_dataset_split_names("squad")

['train', 'validation']

In [4]:
from datasets import load_dataset

dataset_train = load_dataset("squad", split="train")
dataset_test = load_dataset("squad", split="validation")
dataset_train

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [5]:
dataset_train[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [6]:
df_train = pd.DataFrame(dataset_train)

In [7]:
df_train

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,"{'text': ['Oregon'], 'answer_start': [229]}"
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,"{'text': ['Rangoon'], 'answer_start': [414]}"
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,"{'text': ['Minsk'], 'answer_start': [476]}"
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,"{'text': ['1975'], 'answer_start': [199]}"


In [9]:
df2 = pd.json_normalize(df_train['answers'])
df2

,text,answer_start
0,[Saint Bernadette Soubirous],[515]
1,[a copper statue of Christ],[188]
2,[the Main Building],[279]
3,[a Marian place of prayer and reflection],[381]
4,[a golden statue of the Virgin Mary],[92]
...,...,...
87594,[Oregon],[229]
87595,[Rangoon],[414]
87596,[Minsk],[476]
87597,[1975],[199]


In [10]:
df_train= pd.concat([df_train, df2], axis=1)
df_train.drop(["answers"],axis=1,inplace=True)
df_train

,id,title,context,question,text,answer_start
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,[Saint Bernadette Soubirous],[515]
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,[a copper statue of Christ],[188]
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,[the Main Building],[279]
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,[a Marian place of prayer and reflection],[381]
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,[a golden statue of the Virgin Mary],[92]
...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,[Oregon],[229]
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,[Rangoon],[414]
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,[Minsk],[476]
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,[1975],[199]


In [21]:
df_train['text'] = [''.join(map(str, l)) for l in df_train['text']]
df_train['answer_start'] = [''.join(map(str, l)) for l in df_train['answer_start']]
df_train

,id,title,context,question,text,answer_start
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92
...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Oregon,229
87595,5735d259012e2f140011a09e,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Rangoon,414
87596,5735d259012e2f140011a09f,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Minsk,476
87597,5735d259012e2f140011a0a0,Kathmandu,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,1975,199


In [22]:
nltk.download('stopwords')
STOP_WORDS = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
def clean_sentence(val):
    # remove chars that are not letters or numbers, downcase, then remove stop words
    regex = re.compile('([^\s\w]|_)+')
    try:
      sentence = regex.sub('', val).lower()
      sentence = sentence.split(" ")
    except :
      sentence = val
    try :
      for word in list(sentence):
        try:
          if word in STOP_WORDS:
            sentence.remove(word)
        except:
          pass
      sentence = " ".join(sentence)
    except :
      pass

    return sentence

In [24]:
def clean_dataframe(data):
    # drop nans, then apply 'clean_sentence' function to 'context', 'question', 'text', 'answer_start'
    data = data.dropna(how="any")
    for col in ['context', 'question', 'text']:
      data[col] = data[col].apply(clean_sentence)
    return data

In [25]:
data_train= clean_dataframe(df_train)
data_train.head(5)

,id,title,context,question,text,answer_start
0,5733be284776f41900661182,University_of_Notre_Dame,architecturally school catholic character atop...,virgin mary allegedly appear 1858 lourdes france,saint bernadette soubirous,515
1,5733be284776f4190066117f,University_of_Notre_Dame,architecturally school catholic character atop...,front notre dame main building,copper statue christ,188
2,5733be284776f41900661180,University_of_Notre_Dame,architecturally school catholic character atop...,basilica sacred heart notre dame beside structure,main building,279
3,5733be284776f41900661181,University_of_Notre_Dame,architecturally school catholic character atop...,grotto notre dame,marian place prayer reflection,381
4,5733be284776f4190066117e,University_of_Notre_Dame,architecturally school catholic character atop...,sits top main building notre dame,golden statue virgin mary,92


In [26]:
def build_corpus(data):
  #"Creates a list of lists containing words from each sentence"
    corpus = []
    for col in  ['context', 'question', 'text']:
      for sentence in data[col].iteritems():
        word_list = sentence[1].split(" ")
        corpus.append(word_list)
    return corpus

In [27]:
corpus = build_corpus(data_train)
len(corpus)

<ipython-input-26-a3168befa6aa>:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for sentence in data[col].iteritems():


262797

In [28]:
df_test = pd.DataFrame(dataset_test)
df2 = pd.json_normalize(df_test['answers'])
df_test = pd.concat([df_test, df2], axis=1)
df_test.drop(['answers'],axis=1 , inplace=True)


df_test['text'] = [l[0] for l in df_test['text']]
df_test['answer_start'] = [l[0] for l in df_test['answer_start']]
df_test['text'] = df_test['text'].astype(str)

df_test

,id,title,context,question,text,answer_start
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,Denver Broncos,177
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,Carolina Panthers,249
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"Santa Clara, California",403
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,Denver Broncos,177
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,gold,488
...,...,...,...,...,...,...
10565,5737aafd1c456719005744fb,Force,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,kilogram-force,82
10566,5737aafd1c456719005744fc,Force,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,kilopond,114
10567,5737aafd1c456719005744fd,Force,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,slug,274
10568,5737aafd1c456719005744fe,Force,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,kip,712


In [29]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10570 entries, 0 to 10569
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            10570 non-null  object
 1   title         10570 non-null  object
 2   context       10570 non-null  object
 3   question      10570 non-null  object
 4   text          10570 non-null  object
 5   answer_start  10570 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 495.6+ KB


In [30]:
data_test = clean_dataframe(df_test)
data_test

,id,title,context,question,text,answer_start
0,56be4db0acb8001400a502ec,Super_Bowl_50,super bowl 50 american football game determine...,nfl team represented afc super bowl 50,denver broncos,177
1,56be4db0acb8001400a502ed,Super_Bowl_50,super bowl 50 american football game determine...,nfl team represented nfc super bowl 50,carolina panthers,249
2,56be4db0acb8001400a502ee,Super_Bowl_50,super bowl 50 american football game determine...,super bowl 50 take place,santa clara california,403
3,56be4db0acb8001400a502ef,Super_Bowl_50,super bowl 50 american football game determine...,nfl team super bowl 50,denver broncos,177
4,56be4db0acb8001400a502f0,Super_Bowl_50,super bowl 50 american football game determine...,color used emphasize 50th anniversary super bowl,gold,488
...,...,...,...,...,...,...
10565,5737aafd1c456719005744fb,Force,poundforce metric counterpart less commonly us...,metric term less used newton,kilogramforce,82
10566,5737aafd1c456719005744fc,Force,poundforce metric counterpart less commonly us...,kilogramforce sometimes reffered,kilopond,114
10567,5737aafd1c456719005744fd,Force,poundforce metric counterpart less commonly us...,seldom used unit mass metric system,slug,274
10568,5737aafd1c456719005744fe,Force,poundforce metric counterpart less commonly us...,seldom used term unit force equal 1000 pound f...,kip,712


In [20]:
tfidf_vectorizer_question = TfidfVectorizer()
X_train_question = tfidf_vectorizer_question.fit_transform(data_train['question'])
X_test_question = tfidf_vectorizer_question.transform(data_test['question'])

tfidf_vectorizer_context = TfidfVectorizer()
X_train_context = tfidf_vectorizer_context.fit_transform(data_train['context'])
X_test_context = tfidf_vectorizer_context.transform(data_test['context'])

tfidf_vectorizer_title = TfidfVectorizer()
X_train_title = tfidf_vectorizer_title.fit_transform(data_train['context'])
X_test_title = tfidf_vectorizer_title.transform(data_test['context'])



In [40]:
X_train_combined = hstack([X_train_question, X_train_context,X_train_title])
X_test_combined = hstack([X_test_question, X_test_context,X_test_title])

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_combined, data_train['text'])

In [ ]:
predictions = clf.predict(X_test_combined)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(data_test['text'], predictions)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(['Random Forest'], [accuracy])
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.show()